In [1]:
from yahoo_pickem_scraper import YahooPickEm
from confidence_pickem_sim import ConfidencePickEmSimulator
from pickem_integration import convert_yahoo_to_simulator_format, convert_yahoo_picks_to_dataframe
import pandas as pd
import numpy as np

In [2]:
week = 14
league_id = 6207
num_sims = 10000

yahoo = YahooPickEm(week, league_id, cookies_file='cookies.txt')
# simulator = ConfidencePickEmSimulator(num_sims=num_sims)


In [3]:
# yahoo.games
# yahoo.players
yahoo.results

[{'favorite': 'Det', 'underdog': 'GB', 'spread': 3.5, 'winner': 'Det'},
 {'favorite': 'Ten', 'underdog': 'Jax', 'spread': 3.5, 'winner': 'Jax'},
 {'favorite': 'Mia', 'underdog': 'NYJ', 'spread': 6.5, 'winner': 'Mia'},
 {'favorite': 'Min', 'underdog': 'Atl', 'spread': 5.5, 'winner': 'Min'},
 {'favorite': 'NO', 'underdog': 'NYG', 'spread': 5.0, 'winner': 'NO'},
 {'favorite': 'Phi', 'underdog': 'Car', 'spread': 12.5, 'winner': 'Phi'},
 {'favorite': 'Pit', 'underdog': 'Cle', 'spread': 6.5, 'winner': 'Pit'},
 {'favorite': 'TB', 'underdog': 'LV', 'spread': 6.5, 'winner': 'TB'},
 {'favorite': 'Ari', 'underdog': 'Sea', 'spread': 2.5, 'winner': 'Sea'},
 {'favorite': 'Buf', 'underdog': 'LAR', 'spread': 4.5, 'winner': 'LAR'},
 {'favorite': 'SF', 'underdog': 'Chi', 'spread': 4.0, 'winner': 'SF'},
 {'favorite': 'KC', 'underdog': 'LAC', 'spread': 4.0, 'winner': 'KC'},
 {'favorite': 'Cin', 'underdog': 'Dal', 'spread': 5.5, 'winner': 'Cin'}]

In [4]:
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo)
simulator.add_games_from_dataframe(games_df)

games_df

,home_team,away_team,vegas_win_prob,crowd_home_pick_pct,crowd_home_confidence,crowd_away_confidence,week,actual_outcome
0,Det,GB,0.81,0.81,6.9,3.9,14,True
1,Ten,Jax,0.92,0.92,5.1,3.4,14,False
2,Mia,NYJ,0.95,0.95,7.6,3.9,14,True
3,Min,Atl,0.92,0.92,7.6,3.8,14,True
4,NO,NYG,0.90,0.90,5.9,3.3,14,True
5,Phi,Car,0.99,0.99,11.9,4.7,14,True
6,Pit,Cle,0.95,0.95,8.7,3.8,14,True
7,TB,LV,0.98,0.98,9.6,3.9,14,True
8,Ari,Sea,0.65,0.65,3.8,3.7,14,False
9,Buf,LAR,0.94,0.94,7.8,3.9,14,False


In [7]:
picks_df = convert_yahoo_picks_to_dataframe(yahoo, num_sims)

picks_df.loc[picks_df.player.isin(["Firman's Educated Guesses"]),["player","game","picked_home","confidence"]].drop_duplicates()

,player,game,picked_home,confidence
6340000,Firman's Educated Guesses,GB@Det,False,3.0
6350000,Firman's Educated Guesses,Jax@Ten,True,5.0
6360000,Firman's Educated Guesses,NYJ@Mia,True,10.0
6370000,Firman's Educated Guesses,Atl@Min,True,7.0
6380000,Firman's Educated Guesses,NYG@NO,True,6.0
6390000,Firman's Educated Guesses,Car@Phi,True,8.0
6400000,Firman's Educated Guesses,Cle@Pit,True,4.0
6410000,Firman's Educated Guesses,LV@TB,False,1.0
6420000,Firman's Educated Guesses,Sea@Ari,False,2.0
6430000,Firman's Educated Guesses,LAR@Buf,True,9.0
